# Document Classification Project


In [2]:
# set up libraries 
import pandas as pd
import glob
import os
from bs4 import BeautifulSoup
import requests


In [5]:
# Get data
df = pd.DataFrame(columns=['Text', 'Label'])
main_path = r'C:/Users/gobli/Documents/AI CORE/Machine-Learning-Algos/Adarga_NLP/Data/'
end_path = ['Analytical_Report', 'Incident_Report', 'Situation_Report']
for end in end_path:
    path = main_path + end
    all_files = glob.glob(os.path.join(path, "*.txt")) 
    # print(all_files[0])
    data = [[f, end] for f in all_files]
    # print(pd.DataFrame(data, columns=['Text', 'Label']))
    df = df.append(pd.DataFrame(data, columns=['Text', 'Label']))
print(df)



                                                 Text              Label
0   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
1   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
2   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
3   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
4   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
..                                                ...                ...
65  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
66  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
67  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
68  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
69  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report

[157 rows x 2 columns]


In [4]:
docs = pd.read_csv("C:/Users/gobli/Documents/AI CORE/Machine-Learning-Algos/Adarga_NLP/Data/Data.csv")
docs.dropna(inplace=True)
# print(docs)

for i in range(len(docs)):
    try:
        r = requests.get(docs.iloc[i][0], timeout=10)
    except:
        print("Problem:", i)
        continue

    if r.ok:
        print("Sucess:", i, docs.iloc[i][0])
        soup = BeautifulSoup(r.content, 'html.parser')
        with open("data/Reports/" + str(i) + ".txt", 'wt') as f:
            text = ""
            for element in soup.select('p'):
                text += element.text + "\n"
            try:
                f.write(text)
            except:
                print("Write problem:", i)
                continue


Sucess: 0 https://issafrica.org/iss-today/time-to-tackle-violence-against-women-in-mali
Sucess: 1 http://www.understandingwar.org/backgrounder/russia-review-putins-%E2%80%9Cpeacekeepers%E2%80%9D-will-support-russian-wars
Sucess: 2 http://www.understandingwar.org/backgrounder/russian-president-putin-wins-upset-victory-nagorno-karabakh
Sucess: 3 http://www.understandingwar.org/backgrounder/warning-intelligence-update-iran-increases-pressure-us-forces-iraq
Write problem: 3
Sucess: 4 http://www.understandingwar.org/backgrounder/russia-turkey-competition-escalates-across-theaters
Sucess: 5 https://www.middleeasteye.net/news/uk-foreign-aid-cut-impact-middle-east-yemen-syria
Sucess: 6 https://www.middleeasteye.net/opinion/israel-palestine-deal-century-biden-end
Sucess: 7 https://www.middleeasteye.net/news/israel-saudi-arabia-netanyahu-mbs-secret-meeting-meaning
Sucess: 8 https://www.defenseindustrydaily.com/f18-hornet-fleets-keeping-em-flying-02816/
Sucess: 9 https://www.criticalthreats.org/b